In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ford-sentence-classifiaction-dataset/sample_submission.csv
/kaggle/input/ford-sentence-classifiaction-dataset/train_data.csv
/kaggle/input/ford-sentence-classifiaction-dataset/test_data.csv


In [2]:
import nltk
from nltk.corpus import stopwords
import pandas as pd

In [4]:
data = pd.read_csv('/kaggle/input/ford-sentence-classifiaction-dataset/train_data.csv')
data

,Unnamed: 0,Sentence_id,New_Sentence,Type
0,0,GERRES15609,Author and/or Review architecture/design and o...,Responsibility
1,1,PHERES15784,Should be able to develop custom dynamic shape...,Responsibility
2,2,GERREQ10457,Experience in working crosslly with a larger ...,Requirement
3,3,GERSKL27235,"Previous business experience, including but no...",Skill
4,4,HONSSK18415,Delivering fast and right the first time.,SoftSkill
...,...,...,...,...
60110,60110,UAERES18030,"In this position, you will utilize your progr...",Responsibility
60111,60111,GERRES3026,"In addition, this individual will be responsib...",Responsibility
60112,60112,INDSSK5492,Good problem solving skills.,SoftSkill
60113,60113,PHESSK15092,Good Excel knowledge .,SoftSkill


In [8]:
data.dropna(axis=0,
            how='any',
            thresh=None,
            subset=None,
            inplace=True)
data.shape[0]

59002

In [ ]:
stop_words = set(stopwords.words('english'))
data['New_Sentence'] = data['New_Sentence'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [12]:
data_new=data.values.tolist()

In [13]:
datat=[]
target=[]
for i in range(len(data_new)):
    datat.append(data_new[i][2])
    if (data_new[i][3]=='Experience'):
        target.append('1')
    if (data_new[i][3]=='Responsibility'):
        target.append('3')
    if (data_new[i][3]=='Requirement'):
        target.append('2')
    if (data_new[i][3]=='SoftSkill'):
        target.append('5')
    if (data_new[i][3]=='Skill'):
        target.append('4')
    if (data_new[i][3]=='Education'):
        target.append('0')

In [16]:
import math
train_size=60/100*len(datat)
dev_size=80/100*len(datat)
test_size=math.ceil(len(datat))

In [22]:
counts=dict()
zero_counts=dict()
one_counts=dict()
two_counts=dict()
three_counts=dict()
four_counts=dict()
five_counts=dict()

In [23]:
for i in range(math.ceil(train_size)):
 str=datat[i]
 #print(str,i)
 words = str.split(' ')
 for word in words:
    if word in counts:
        counts[word] += 1
    else:
        counts[word] = 1

In [24]:
c0=c1=c2=c3=c4=c5=0
for i in range(math.ceil(train_size)):
    if(target[i]=='0'):
        c0+=1
    if(target[i]=='1'):
        c1+=1
    if(target[i]=='2'):
        c2+=1
    if(target[i]=='3'):
        c3+=1
    if(target[i]=='4'):
        c4+=1
    if(target[i]=='5'):
        c5+=1

In [25]:
for i in range(math.ceil(train_size)):
    str=datat[i]
    words = str.split(' ')
    if(target[i]=='0'):
     for word in words:
        if word in zero_counts:
            zero_counts[word] += 1
        else:
            zero_counts[word] = 1
    if(target[i]=='1'):
        for word in words:
            if word in one_counts:
                one_counts[word] += 1
            else:
                one_counts[word] = 1
    if(target[i]=='2'):
        for word in words:
            if word in two_counts:
                two_counts[word] += 1
            else:
                two_counts[word] = 1
    if(target[i]=='3'):
        for word in words:
            if word in three_counts:
                three_counts[word] += 1
            else:
                three_counts[word] = 1
    if(target[i]=='4'):
        for word in words:
            if word in four_counts:
                four_counts[word] += 1
            else:
                four_counts[word] = 1
    if(target[i]=='5'):
        for word in words:
            if word in five_counts:
                five_counts[word] += 1
            else:
                five_counts[word] = 1

In [26]:
def find_cp(str,cdict,c):
    words = str.split(' ')
    calc=[]
    count=0
    den=math.ceil(train_size)
    #prob=c/47202
    for word in words:
        if word in cdict.keys() :
            #if(cdict[word]>5):
              #prob=prob*(cdict[word]/c)
              calc.append(cdict[word])
        else:
            #continue
            #prob=prob*(1/c)
            calc.append(1)
            count+=1
    den=den+count
    c=c+count
    prob=c/den
    for i in range(len(calc)):
        prob=prob*(calc[i]/c)

    return (prob)

In [27]:
acc=0
cp0=cp1=cp2=cp4=cp5=hpc=0
pc='3'
for i in range(math.ceil(train_size),math.ceil(dev_size)):
    str=datat[i]
    #words = str.split(' ')
    cp0=find_cp(str,zero_counts,c0)
    cp1=find_cp(str,one_counts,c1)
    cp2=find_cp(str,two_counts,c2)
    cp3=find_cp(str,three_counts,c3)
    cp4=find_cp(str,four_counts,c4)
    cp5=find_cp(str,five_counts,c5)
    hpc=max(cp0,cp1,cp2,cp3,cp4,cp5)
    if(hpc==cp0):
        pc='0'
    if(hpc==cp1):
        pc='1'
    if(hpc==cp2):
        pc='2'
    if(hpc==cp3):
        pc='3'
    if(hpc==cp4):
        pc='4'
    if(hpc==cp5):
        pc='5'
    #print(hpc)
    if(target[i]==pc):
        acc+=1

print('Accuracy upon predicting the Dev data set is ',acc/(math.ceil(dev_size)-math.ceil(train_size)))

Accuracy upon predicting the Dev data set is  0.6844067796610169


In [28]:
acc2=0
cp0=cp1=cp2=cp4=cp5=hpc=0
for i in range(math.ceil(dev_size),math.ceil(test_size)):
    str=datat[i]
    #words = str.split(' ')
    cp0=find_cp(str,zero_counts,c0)
    cp1=find_cp(str,one_counts,c1)
    cp2=find_cp(str,two_counts,c2)
    cp3=find_cp(str,three_counts,c3)
    cp4=find_cp(str,four_counts,c4)
    cp5=find_cp(str,five_counts,c5)
    hpc=max(cp0,cp1,cp2,cp3,cp4,cp5)
    if(hpc==cp0):
        pc='0'
    if(hpc==cp1):
        pc='1'
    if(hpc==cp2):
        pc='2'
    if(hpc==cp3):
        pc='3'
    if(hpc==cp4):
        pc='4'
    if(hpc==cp5):
        pc='5'
    #print(hpc)
    if(target[i]==pc):
        acc2+=1

print('Accuracy upon predicting test data is ',acc2/(math.ceil(test_size)-math.ceil(dev_size)))

Accuracy upon predicting test data is  0.6724576271186441


In [32]:
acc3=0
cp0=cp1=cp2=cp4=cp5=hpc=0
for i in range(math.ceil(test_size)):
    str=datat[i]
    #words = str.split(' ')
    cp0=find_cp(str,zero_counts,c0)
    cp1=find_cp(str,one_counts,c1)
    cp2=find_cp(str,two_counts,c2)
    cp3=find_cp(str,three_counts,c3)
    cp4=find_cp(str,four_counts,c4)
    cp5=find_cp(str,five_counts,c5)
    hpc=max(cp0,cp1,cp2,cp3,cp4,cp5)
    if(hpc==cp0):
        pc='0'
    if(hpc==cp1):
        pc='1'
    if(hpc==cp2):
        pc='2'
    if(hpc==cp3):
        pc='3'
    if(hpc==cp4):
        pc='4'
    if(hpc==cp5):
        pc='5'
    #print(hpc)
    if(target[i]==pc):
        acc3+=1

print('Accuracy upon predicting complete dataset is ',acc3/(math.ceil(test_size)))

Accuracy upon predicting complete dataset is  0.7092979898986475
